In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import spacy
from spacy.lang.fr import French
from spacy.tokenizer import Tokenizer
from spacy.tokens import Doc
from spacy.vocab import Vocab
from spacy.matcher import Matcher
from spacy.matcher import PhraseMatcher
from textblob import TextBlob
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import re

/anaconda3/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
measurements = {"g": ["gr.", "gr", "g", "g.", "grams", "gramme", "GR"], "kg": ["kg.", "kg", "kilogramme", "vrac", "Vrac"], "mg": ["mg.", "mg", "miligramme"],
                "ml": ["ml.", "ml", "milliliters", "mililitre", "ML"], "cl": ["cl.", "cl", "centilitre"], "l": ["l.", "l", "litre"],
                "cuillère à soupe": ["c. à soupe", "cuillère à soupe", "cs", "cas", "càs", "c. à table", "cuillère à table"],
                "cuillère à café": ["c. à café", "cuillère à café", "cc", "cac", "càc", "c. à thé", "cuillère à thé"],
                "tasse": ["tasse"], "bol": ["bol"], "verre": ["verre"], "filet": ["filet"], "zeste": ["zeste"], "pièce": ["pièce", "unité", "ampollas", "set"],
                "noisette": ["noisette"], "noix": ["noix"], "pincée": ["pincée"], "pointe":["pointe"], "poignée": ["poignée"],
                "feuille": ["feuille"], "branche": ["branche"], "gousse": ["gousse"], "tranche": ["tranche"], "cube": ["cube"],
                "boîte": ["boîte"], "barquette": ["barquette"], "pot": ["pot"], "bâtonnet": ["bâtonnet"], "boule": ["boule"],
                "rouleau": ["rouleau"], "p": ["p"]}

units = [item for sublist in measurements.values() for item in sublist]

quantities = { "½": 0.5, "1/2": 0.5, "1½": 1.5, "1 1/2": 0.5, "⅓": 0.33, "1/3": 0.33, "¼": 0.25, "1/5": 0.2, "¼": 0.25, "un demi": 0.5, "une demi": 0.5,"un et demi": 1.5,
              "tiers": 0.33, "quart": 0.25, "zero": 0, "deux": 2, "trois": 3, "quatre": 4,"cinq": 5, "six": 6, "sept": 7,
              "huit": 8, "neuf": 9, "dix": 10, "onze": 11, "douze": 12, "treize": 13, "quatorze": 14, "quinze": 15, "seize": 16,
              "dix-sept": 17, "dix-huit": 18, "dix-neuf": 19, "vingt": 20, "trente": 30, "quarante": 40, "cinquante": 50,
              "soixante": 60, "soixante-dix": 70, "quatre-vingt": 80, "quatre-vingt-dix": 90, "dizaine": 10, "une dizaine": 10,
              "douzaine": 12, "une douzaine": 12, "demi-douzaine": 6, "une demi-douzaine": 6, "vingtaine": 20, "une vingtaine": 20,
              "trentaine": 30, "quarantaine": 40, "cinquantaine": 50, "centaine": 100, "une centaine": 100, "cent": 100,
              "un": 1, "une": 1, "vrac": 1}

char_list = [i for i in "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZéèàâêïù▪️"]


In [3]:
web_product_data = pd.read_csv ('/Users/vincentsalamand/Downloads/products_biocbon.csv')
print(len(web_product_data))

880


In [12]:
data = pd.DataFrame()
data['quantity_match'] = ""
data['unit_match'] = ""
data['price_match'] = ""
data['price_per_unit_match'] = ""
data = pd.concat([data, web_product_data], axis=1)
data['shelter'] = ""

In [17]:
data.head()

,quantity_match,unit_match,price_match,price_per_unit_match,shelter_parent,shelter_child,brand,description1,description,price,price_per_unit,store,image_url,url,ean,shelter
0,0.125,NaN,2.60,20.80,Fruits et légumes,Légumes,NaN,"Mesclun bio, barquette 125g","Mesclun bio, barquette 125g",2.60,20.80 €/kg,Bio c' Bon,https://commande.bio-c-bon.fr/media/catalog/pr...,https://commande.bio-c-bon.fr/jeunes-pousses-m...,3660,
1,1.000,NaN,3.50,3.50,Fruits et légumes,Légumes,NaN,Chou-fleur bio (à la pièce),Chou-fleur bio (à la pièce),3.50,3.50€/ pièce,Bio c' Bon,https://commande.bio-c-bon.fr/media/catalog/pr...,https://commande.bio-c-bon.fr/chou-fleur.html,3665,
2,1.000,NaN,1.80,1.80,Fruits et légumes,Légumes,NaN,"Persil plat bio, botte (à la pièce)","Persil plat bio, botte (à la pièce)",1.80,1.80€/ pièce,Bio c' Bon,https://commande.bio-c-bon.fr/media/catalog/pr...,https://commande.bio-c-bon.fr/persil-botte.html,3687,
3,0.250,NaN,1.90,7.60,Fruits et légumes,Légumes,NaN,"Tomates Cerises bio, barquette 250g","Tomates Cerises bio, barquette 250g",1.90,7.60 €/kg,Bio c' Bon,https://commande.bio-c-bon.fr/media/catalog/pr...,https://commande.bio-c-bon.fr/tomate-cerise-ba...,3698,
4,0.500,NaN,1.17,2.34,Fruits et légumes,Légumes,NaN,Oignons jaunes bio (500 g),Oignons jaunes bio (500 g),1.17,2.34 €/kg,Bio c' Bon,https://commande.bio-c-bon.fr/media/catalog/pr...,https://commande.bio-c-bon.fr/oignon-jaune.html,6307,


In [15]:
# clean prices
data['price_match'] = data.price
data['price_per_unit_match'] = data.price_per_unit.apply(lambda x: float(x.replace('€/',' ').split()[0]))
data.quantity_match = round(data.price_match / data.price_per_unit_match, 3)


In [20]:
# match description to find unit
nlp = French()

def convert_unit(value):
    results = []
    result = [unit for unit, item in measurements.items() if value in item]
    if len(result) > 0:
        return result[0]

measurement_matcher = PhraseMatcher(nlp.vocab, attr="LOWER")
measurement_patterns = list(nlp.tokenizer.pipe(list(units)))
measurement_matcher.add("MEASUREMENT_PATTERN", None, *measurement_patterns)
tokenizer = Tokenizer(nlp.vocab)

measurement_parser = []
price_per_unit = list(nlp.pipe(data.price_per_unit))

for line in price_per_unit:
    line = tokenizer(re.sub('[0-9]', ' ', line.text.replace('€/',' ')))
    matches = measurement_matcher(line)
    elements = []
    if len(matches) > 0:
        for match_id, start, end in matches:
            span = line[start:end]
            elements.append(span.text.lower())
        measurement_parser.append(convert_unit(elements[0]))
    else:
        text_blob_object = TextBlob(line.text)
        singular_line = ' '.join(text_blob_object.words.singularize())
        matches = measurement_matcher(tokenizer(singular_line))
        if len(matches) > 0:
            for match_id, start, end in matches:
                span = tokenizer(singular_line)[start:end]
                elements.append(span.text.lower())
            measurement_parser.append(convert_unit(elements[0]))
        else:
            plural_line = ' '.join(text_blob_object.words.pluralize())
            matches = measurement_matcher(tokenizer(plural_line))
            if len(matches) > 0:
                for match_id, start, end in matches:
                    span = tokenizer(plural_line)[start:end]
                    elements.append(span.text.lower())
                measurement_parser.append(convert_unit(elements[0]))
            else:
                 measurement_parser.append("")

data['unit_match'] = measurement_parser

In [22]:
# get shelter list

for index, product in data.iterrows():       
    shelter = []
    shelter.append(data.loc[index].shelter_parent)    
    shelter.append(data.loc[index].shelter_child)
    #if "Surgelés" in shelter:
     #   print(product)
    data.at[index,'shelter'] = shelter

In [24]:
export_csv = data.to_csv (r'/Users/vincentsalamand/Downloads/biocbon_catalog.csv', index = None, header=True)


In [23]:
data

,quantity_match,unit_match,price_match,price_per_unit_match,shelter_parent,shelter_child,brand,description1,description,price,price_per_unit,store,image_url,url,ean,shelter
0,0.125,kg,2.60,20.80,Fruits et légumes,Légumes,NaN,"Mesclun bio, barquette 125g","Mesclun bio, barquette 125g",2.60,20.80 €/kg,Bio c' Bon,https://commande.bio-c-bon.fr/media/catalog/pr...,https://commande.bio-c-bon.fr/jeunes-pousses-m...,3660,"[Fruits et légumes, Légumes]"
1,1.000,pièce,3.50,3.50,Fruits et légumes,Légumes,NaN,Chou-fleur bio (à la pièce),Chou-fleur bio (à la pièce),3.50,3.50€/ pièce,Bio c' Bon,https://commande.bio-c-bon.fr/media/catalog/pr...,https://commande.bio-c-bon.fr/chou-fleur.html,3665,"[Fruits et légumes, Légumes]"
2,1.000,pièce,1.80,1.80,Fruits et légumes,Légumes,NaN,"Persil plat bio, botte (à la pièce)","Persil plat bio, botte (à la pièce)",1.80,1.80€/ pièce,Bio c' Bon,https://commande.bio-c-bon.fr/media/catalog/pr...,https://commande.bio-c-bon.fr/persil-botte.html,3687,"[Fruits et légumes, Légumes]"
3,0.250,kg,1.90,7.60,Fruits et légumes,Légumes,NaN,"Tomates Cerises bio, barquette 250g","Tomates Cerises bio, barquette 250g",1.90,7.60 €/kg,Bio c' Bon,https://commande.bio-c-bon.fr/media/catalog/pr...,https://commande.bio-c-bon.fr/tomate-cerise-ba...,3698,"[Fruits et légumes, Légumes]"
4,0.500,kg,1.17,2.34,Fruits et légumes,Légumes,NaN,Oignons jaunes bio (500 g),Oignons jaunes bio (500 g),1.17,2.34 €/kg,Bio c' Bon,https://commande.bio-c-bon.fr/media/catalog/pr...,https://commande.bio-c-bon.fr/oignon-jaune.html,6307,"[Fruits et légumes, Légumes]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
875,0.135,kg,3.19,23.63,Sans gluten et régimes spéciaux,Sans gluten sucré,NaN,Sablés pur beurre 135g,Sablés pur beurre 135g,3.19,23.63 €/kg,Bio c' Bon,https://commande.bio-c-bon.fr/media/catalog/pr...,https://commande.bio-c-bon.fr/sables-pur-beurr...,3760154261609,"[Sans gluten et régimes spéciaux, Sans gluten ..."
876,0.180,kg,5.18,28.78,Sans gluten et régimes spéciaux,Sans gluten sucré,NaN,Tablette noir 70% 180g,Tablette noir 70% 180g,5.18,28.78 €/kg,Bio c' Bon,https://commande.bio-c-bon.fr/media/catalog/pr...,https://commande.bio-c-bon.fr/tablette-noir-70...,3258590000406,"[Sans gluten et régimes spéciaux, Sans gluten ..."
877,0.200,kg,4.65,23.26,Sans gluten et régimes spéciaux,Sans gluten sucré,NaN,Biscuit sésame figue 200g,Biscuit sésame figue 200g,4.65,23.26 €/kg,Bio c' Bon,https://commande.bio-c-bon.fr/media/catalog/pr...,https://commande.bio-c-bon.fr/biscuit-sesame-f...,3760031830300,"[Sans gluten et régimes spéciaux, Sans gluten ..."
878,0.450,kg,5.39,11.98,Sans gluten et régimes spéciaux,Sans gluten sucré,NaN,Muesli fruits du soleil 450g,Muesli fruits du soleil 450g,5.39,11.98 €/kg,Bio c' Bon,https://commande.bio-c-bon.fr/media/catalog/pr...,https://commande.bio-c-bon.fr/muesli-fruits-du...,3111900350632,"[Sans gluten et régimes spéciaux, Sans gluten ..."
